<a href="https://colab.research.google.com/github/jideilori/semrush_scrapper/blob/main/scrapper_semrush.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import csv

pd.set_option('display.max_columns', None)

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:127.0) Gecko/20100101 Firefox/127.0',
    'Accept': '*',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Content-Type': 'text/plain;charset=UTF-8',
    'Origin': 'https://www.semrush.com',
    'Connection': 'keep-alive',
    'Referer': 'https://www.semrush.com/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'Priority': 'u=1',
}

## Scrape all industries to get column names and categories

---



In [55]:
import requests
import json
from bs4 import BeautifulSoup

def fetch_trending_website_data(url, headers):
    """
    Fetches and parses trending website data from a given URL.

    This function retrieves HTML content from a specified URL, searches for a specific
    script tag containing a JavaScript variable `window.__PRELOADED_STATE__` with JSON data,
    and parses it into a Python dictionary. It also extracts relevant column names for data processing.

    Args:
        url (str): The URL to fetch data from.
        headers (dict): HTTP headers to use for the GET request.

    Returns:
        tuple: A tuple containing:
            - list : Categories extracted from semrush url.
            - list: Column names extracted from the JSON data.

    Example:
        ```python
        url = 'https://www.semrush.com/trending-websites/us/all'
        headers = {'User-Agent': 'your-user-agent'}
        categories, column_names = fetch_trending_website_data(url, headers)
        ```

    Notes:
        - The function searches for the script tag containing `window.__PRELOADED_STATE__` and extracts the JSON data.
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    search_str = 'window.__PRELOADED_STATE__ = '
    search_str_len = len(search_str)

    scripts = soup.find_all('script')

    # Get index of the script tag containing 'window.__PRELOADED_STATE__ = '
    scripts_text_index = next(
        (index for index, script in enumerate(scripts) if script.text.strip().startswith(search_str)),
        None
    )

    if scripts_text_index is None:
        print("Could not find the required script tag.")
        return None, []

    text = scripts[scripts_text_index].text.strip()
    json_str = text[search_str_len:-1]

    # Parse the JSON data
    try:
        json_data = json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None, []

    # Extract column names
    if 'data' in json_data and 'domains' in json_data['data'] and json_data['data']['domains']:
        column_names = list(json_data['data']['domains'][0].keys())
        column_names.append('category')
    else:
        column_names = []

    return json_data['categories'], column_names


url = 'https://www.semrush.com/trending-websites/us/all'
categories, column_names = fetch_trending_website_data(url, headers)

111
['domain_name', 'total_traffic', 'desktop_percent', 'desktop_traffic', 'mobile_percent', 'mobile_traffic', 'mom', 'yoy', 'top_source', 'category']


## Scrape domains

In [6]:
# Function to extract JSON from text
def extract_json_from_text(cat_soup):
    cat_scripts = cat_soup.find_all('script')
    search_str = 'window.__PRELOADED_STATE__ = '
    cat_search_str_len = len(search_str)

    for script in cat_scripts:
        if script.text.strip().startswith(search_str):
            text = script.text.strip()
            cat_json_str = text[cat_search_str_len:-1]

            try:
                cat_json_data = json.loads(cat_json_str)
                return cat_json_data
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                return None
    print("Could not find the required script tag.")
    return None


In [7]:
# Runs in 40 seconds
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import concurrent.futures
import time

#
all_df = pd.DataFrame(columns=column_names)




# Function to scrape a category
def scrape_category(category):
    category_url = f'https://www.semrush.com/trending-websites/us/{category}'
    cat_response = requests.get(category_url, headers=headers)
    time.sleep(1)  # Be respectful of server limits
    cat_soup = BeautifulSoup(cat_response.text, 'html.parser')
    cat_json_data = extract_json_from_text(cat_soup)

    if cat_json_data and 'data' in cat_json_data and 'domains' in cat_json_data['data']:
        cat_df = pd.DataFrame(cat_json_data['data']['domains'])
        cat_df['category'] = category
        return cat_df
    else:
        return pd.DataFrame()  # Return an empty DataFrame if JSON is missing or incomplete

# Concurrently scrape all categories
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(scrape_category, category) for category in categories]
    all_df = pd.concat([future.result() for future in concurrent.futures.as_completed(futures)], ignore_index=True)

# Save to CSV
all_df.to_csv('semrush_listed_sites_concurrent.csv', index=False)

In [8]:
# preview_data
pd.read_csv('/content/semrush_listed_sites_concurrent.csv')

,domain_name,total_traffic,desktop_percent,desktop_traffic,mobile_percent,mobile_traffic,mom,yoy,top_source,category
0,uworld.com,10061449,97.620542,9822041,2.379458,239408,-0.960057,-7.524990,direct,accounting-and-auditing
1,sharefile.com,5157785,90.241470,4654461,9.758530,503324,-28.410870,1.497492,direct,accounting-and-auditing
2,freetaxusa.com,4706463,44.103396,2075710,55.896604,2630753,-75.268360,25.183290,direct,accounting-and-auditing
3,deloitte.com,3862722,86.295131,3333341,13.704869,529381,-1.286360,13.783615,direct,accounting-and-auditing
4,mip.com,2411077,7.075884,170605,92.924116,2240472,-12.416770,11387.501000,direct,accounting-and-auditing
...,...,...,...,...,...,...,...,...,...,...
11095,drudgereport.com,98921020,26.216331,25933462,73.783669,72987558,-7.095231,-23.218384,direct,all
11096,adobe.com,98630173,64.544602,63660453,35.455398,34969720,0.219971,-9.724714,direct,all
11097,genius.com,97889688,32.748744,32057643,67.251256,65832045,4.243665,-1.350669,search,all
11098,weather.gov,96587257,33.816601,32662527,66.183399,63924730,-11.347692,-11.640446,direct,all


In [9]:
# copy to drive
!cp /content/semrush_listed_sites_concurrent.csv /content/drive/MyDrive/mldata/mylane

## Scrape Sites data

### Concurrent scrapping

In [11]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import concurrent.futures
import time
import csv
import random

# Scraping takes around 3 hours

# Function to extract site data
def get_site_data(domain_name):
    url = f'https://www.semrush.com/website/{domain_name}/overview/?source=trending-websites'
    random_number = random.randint(2, 4)
    time.sleep(random_number)  # Be respectful of server limits
    site_response = requests.get(url, headers=None)
    site_soup = BeautifulSoup(site_response.text, 'html.parser')

    script_tag = site_soup.find('script', id='__NEXT_DATA__')

    if not script_tag or not script_tag.string:
        print(f"No valid script tag found for {domain_name}")
        return None

    data = json.loads(script_tag.string)



    try:
      site_data = data
    except KeyError as e:
        print(f"Error: {e}")
        site_data = None

    try:
        page_props = data['props']['pageProps']['page']
    except KeyError as e:
        print(f"Error: {e}")
        page_props = {}

    try:
        categories = [i['name'] for i in page_props.get('categories', [])]
    except KeyError as e:
        print(f"Error: {e}")
        categories = []

    try:
        global_rank = page_props['trafficStats']['globalRank']['value']
    except KeyError as e:
        print(f"Error: {e}")
        global_rank = None

    try:
        country_rank = page_props['trafficStats']['countryRank']['value']
    except KeyError as e:
        print(f"Error: {e}")
        country_rank = None

    try:
        visits_no = page_props['trafficStats']['visits']['value']
    except KeyError as e:
        print(f"Error: {e}")
        visits_no = None

    try:
        authority_score = page_props['trafficStats']['authorityScore']['value']
    except KeyError as e:
        print(f"Error: {e}")
        authority_score = None

    try:
        pages_per_visit = page_props['visitorEngagement']['pagesPerVisit']['value']
    except KeyError as e:
        print(f"Error: {e}")
        pages_per_visit = None

    try:
        time_on_site = (page_props['visitorEngagement']['timeOnSite']['value'] / 60)
    except KeyError:
        time_on_site = None

    try:
        value_diff_percent = page_props['visitorEngagement']['visits']['valueDiffPercent']
    except KeyError:
        value_diff_percent = None

    try:
        calc = time_on_site * value_diff_percent
    except (TypeError, KeyError) as e:
        print(f"Error: {e}")
        calc = None

    try:
        avg_visit_duration = round((time_on_site - calc), 2)
    except KeyError as e:
        print(f"Error: {e}")
        avg_visit_duration = None

    try:
        bounce_rate = page_props['visitorEngagement']['bounceRate']['value'] * 100
    except KeyError as e:
        print(f"Error: {e}")
        bounce_rate = None

    try:
        traffic_by_country = page_props['traffic_by_country']
    except KeyError as e:
        print(f"Error: {e}")
        traffic_by_country = None

    try:
        traffic_by_device = page_props['traffic_by_device']['traffic_by_device_history'][0]
    except (KeyError, IndexError) as e:
        print(f"Error: {e}")
        traffic_by_device = None

    try:
        desktop_device = round(((traffic_by_device['desktop_visits'] / traffic_by_device['visits']) * 100), 2)
    except KeyError as e:
        print(f"Error: {e}")
        desktop_device = None

    try:
        mobile_device = round(((traffic_by_device['mobile_visits'] / traffic_by_device['visits']) * 100), 2)
    except KeyError as e:
        print(f"Error: {e}")
        mobile_device = None

    try:
        competitors = page_props['competitors']
    except KeyError as e:
        print(f"Error: {e}")
        competitors = None

    try:
        organic_traffic = page_props['traffic_overview']['traffic_organic']['value']
    except KeyError as e:
        print(f"Error: {e}")
        organic_traffic = None

    try:
        paid_traffic = page_props['traffic_overview']['traffic_paid']['value']
    except KeyError as e:
        print(f"Error: {e}")
        paid_traffic = None

    try:
        backlinks = page_props['backlinkAnalytics']['backlinks']['value']
    except KeyError as e:
        print(f"Error: {e}")
        backlinks = None

    try:
        referring_domains = page_props['backlinkAnalytics']['referringDomains']['value']
    except KeyError as e:
        print(f"Error: {e}")
        referring_domains = None

    result = {
        'domain_name': domain_name,
        'global_rank': global_rank,
        'country_rank': country_rank,
        'visits_no': visits_no,
        'authority_score': authority_score,
        'pages_per_visit': pages_per_visit,
        'avg_visit_duration': avg_visit_duration,
        'bounce_rate': bounce_rate,
        'referring_domains': referring_domains,
        'backlinks': backlinks,
        'traffic_by_country': traffic_by_country,
        'traffic_by_device': traffic_by_device,
        'desktop_device': desktop_device,
        'mobile_device': mobile_device,
        'competitors': competitors,
        'organic_traffic': organic_traffic,
        'paid_traffic': paid_traffic,
        'categories': categories,
        'site_data': site_data
    }

    return {key: str(value) for key, value in result.items()}


# Read the CSV file containing domain names
sites_list = pd.read_csv('semrush_listed_sites_concurrent.csv')
domains = sites_list['domain_name']

site_column_names = [
    'domain_name', 'global_rank', 'country_rank', 'visits_no',
    'authority_score', 'pages_per_visit', 'avg_visit_duration',
    'bounce_rate', 'referring_domains', 'backlinks', 'traffic_by_country',
    'traffic_by_device', 'desktop_device', 'mobile_device',
    'competitors', 'organic_traffic', 'paid_traffic', 'categories', 'site_data'
]

# Function to process and save site data concurrently
def process_and_save_data():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(get_site_data, domain): domain for domain in domains}

        with open('site_data.csv', 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=site_column_names)
            writer.writeheader()

            for future in concurrent.futures.as_completed(futures):
                domain = futures[future]
                try:
                    site_result = future.result()
                    if site_result:
                        writer.writerow(site_result)
                except Exception as e:
                    print(f"Error processing {domain}: {e}")

# Run the data processing
process_and_save_data()


Error processing vlr.gg: unsupported operand type(s) for /: 'NoneType' and 'int'
Error: unsupported operand type(s) for *: 'float' and 'NoneType'
Error processing maranastockyards.com: unsupported operand type(s) for -: 'float' and 'NoneType'
No valid script tag found for rainharvest.com
No valid script tag found for supplyone.com
No valid script tag found for reusabletranspack.com
No valid script tag found for nglantz.com
No valid script tag found for harcofittings.com
No valid script tag found for resource-recycling.com
No valid script tag found for shapesplastics.com
No valid script tag found for adobe.com
No valid script tag found for flickr.com
No valid script tag found for freepik.com
No valid script tag found for gettyimages.com
No valid script tag found for piedmontplastics.com
No valid script tag found for istockphoto.com
No valid script tag found for pixieset.com
No valid script tag found for smugmug.com
No valid script tag found for matterport.com
No valid script tag found f

In [13]:
!cp /content/site_data.csv /content/drive/MyDrive/mldata/mylane

In [14]:
site_df = pd.read_csv('/content/site_data.csv')

In [22]:
site_df.domain_name.duplicated().sum()

1501

In [ ]:
# Get failed websites and rescrape

In [23]:
failed_websites = [
    "rainharvest.com",
    "supplyone.com",
    "reusabletranspack.com",
    "nglantz.com",
    "harcofittings.com",
    "resource-recycling.com",
    "shapesplastics.com",
    "adobe.com",
    "flickr.com",
    "freepik.com",
    "gettyimages.com",
    "piedmontplastics.com",
    "istockphoto.com",
    "pixieset.com",
    "smugmug.com",
    "matterport.com",
    "alamy.com",
    "unsplash.com",
    "dpreview.com",
    "pixabay.com",
    "canon.com",
    "vecteezy.com",
    "dreamstime.com",
    "adorama.com",
    "depositphotos.com",
    "petapixel.com",
    "gradimages.com",
    "mpb.com",
    "pngtree.com",
    "shootproof.com",
    "500px.com",
    "wallpaperflare.com",
    "mpix.com",
    "pngwing.com"
]


In [24]:
def get_site_data(url):

    time.sleep(1)
    site_response = requests.get(url, headers=None)
    site_soup = BeautifulSoup(site_response.text, 'html.parser')


    script_tag = site_soup.find('script', id='__NEXT_DATA__')

    # Extract the content from the script tag
    if script_tag:
        script_content = script_tag.string
        data = json.loads(script_content)


    site_data = data
    data = data['props']['pageProps']['page']

    categories = [i['name'] for i in data['categories']]
    global_rank = data['trafficStats']['globalRank']['value']
    country_rank = data['trafficStats']['countryRank']['value']
    visits_no = data['trafficStats']['visits']['value']
    authority_score = data['trafficStats']['authorityScore']['value']
    pages_per_visit = data['visitorEngagement']['pagesPerVisit']['value']

    # ___average visit duration___
    time_on_site = (data['visitorEngagement']['timeOnSite']['value']/60)
    value_diff_percent = data['visitorEngagement']['visits']['valueDiffPercent']
    calc = time_on_site*value_diff_percent
    avg_visit_duration = round((time_on_site - calc),2)

    bounce_rate = data['visitorEngagement']['bounceRate']['value'] * 100

    traffic_by_country = data['traffic_by_country']
    traffic_by_device = data['traffic_by_device']['traffic_by_device_history'][0] #The most current history - [0]
    desktop_device = round(((traffic_by_device['desktop_visits']/traffic_by_device['visits']) * 100),2)
    mobile_device = round(((traffic_by_device['mobile_visits']/traffic_by_device['visits']) * 100),2)

    competitors = data['competitors']

    organic_traffic = data['traffic_overview']['traffic_organic']['value']
    paid_traffic = data['traffic_overview']['traffic_paid']['value']

    backlinks = data['backlinkAnalytics']['backlinks']['value']
    referring_domains = data['backlinkAnalytics']['referringDomains']['value']


    result = {
          'domain_name':domain_name,
          'global_rank':global_rank,
          'country_rank':country_rank,
          'visits_no':visits_no,
          'authority_score':authority_score,
          'pages_per_visit':pages_per_visit,
          'avg_visit_duration':avg_visit_duration,
          'bounce_rate':bounce_rate,
          'referring_domains':referring_domains,
          'backlinks':backlinks,
          'traffic_by_country':traffic_by_country,
          'traffic_by_device': traffic_by_device,
          'desktop_device':desktop_device,
          'mobile_device':mobile_device,
          'competitors':competitors,
          'organic_traffic':organic_traffic,
          'paid_traffic':paid_traffic,
          'categories':categories,
          'site_data': site_data
      }




    # ____________Output____________

    # convert all to string
    result = {key: str(value) for key, value in result.items()}

    return result


sites_details = []

domains = failed_websites
site_column_names = ['domain_name', 'global_rank', 'country_rank', 'visits_no',\
                     'authority_score', 'pages_per_visit', 'avg_visit_duration',\
                     'bounce_rate', 'referring_domains', 'backlinks', 'traffic_by_country', \
                     'traffic_by_device', 'desktop_device', 'mobile_device', \
                     'competitors', 'organic_traffic', 'paid_traffic', 'categories', 'site_data']

with open('failed_websites_semrush_sites_data.csv', 'w', newline='') as csvfile:
    # Create a CSV writer object
    writer = csv.DictWriter(csvfile, fieldnames=site_column_names)

    # Write the header row
    writer.writeheader()

    for index,domain_name in enumerate(domains):
      url = f'https://www.semrush.com/website/{domain_name}/overview/?source=trending-websites'
      site_result = get_site_data(url)


      writer.writerow(site_result)

      if index%25==0:
        print(index)

0
25


In [26]:
# merge dataframes
f_df = pd.read_csv('/content/failed_websites_semrush_sites_data.csv')
s_df = pd.read_csv('site_data.csv')

final_df = pd.concat([f_df, s_df], ignore_index=True)
final_df

,domain_name,global_rank,country_rank,visits_no,authority_score,pages_per_visit,avg_visit_duration,bounce_rate,referring_domains,backlinks,traffic_by_country,traffic_by_device,desktop_device,mobile_device,competitors,organic_traffic,paid_traffic,categories,site_data
0,rainharvest.com,20,24,2234905088,100.0,3.1522,13.97,52.80,379479.0,1.994620e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22349...",80.18,19.82,"{'display_date': '2024-05-15', 'competitors': ...",395662082.0,1842.0,"['Entertainment', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
1,supplyone.com,20,24,2234905088,100.0,3.1522,13.97,52.80,379479.0,1.994620e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22349...",80.18,19.82,"{'display_date': '2024-05-15', 'competitors': ...",395662082.0,1842.0,"['Entertainment', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
2,reusabletranspack.com,20,24,2234905088,100.0,3.1522,13.97,52.80,379479.0,1.994620e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22349...",80.18,19.82,"{'display_date': '2024-05-15', 'competitors': ...",395662082.0,1842.0,"['Entertainment', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
3,nglantz.com,20,24,2234905088,100.0,3.1522,13.97,52.80,379479.0,1.994620e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22349...",80.18,19.82,"{'display_date': '2024-05-15', 'competitors': ...",395662082.0,1842.0,"['Entertainment', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
4,harcofittings.com,20,24,2234905088,100.0,3.1522,13.97,52.80,379479.0,1.994620e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22349...",80.18,19.82,"{'display_date': '2024-05-15', 'competitors': ...",395662082.0,1842.0,"['Entertainment', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11093,forbes.com,283,106,170285922,100.0,1.4842,4.49,76.53,1508842.0,4.330822e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 17028...",30.32,69.68,"{'display_date': '2024-05-15', 'competitors': ...",98133930.0,541805.0,"['Newspapers', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
11094,adobe.com,95,109,436231535,100.0,3.1437,7.37,57.58,2013681.0,1.337587e+09,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 43623...",63.25,36.75,"{'display_date': '2024-05-15', 'competitors': ...",179383533.0,4320379.0,"['Computer Software and Development', 'Online ...",{'props': {'pageProps': {'page': {'actualDate'...
11095,genius.com,155,111,274113391,94.0,2.5551,8.19,69.83,251976.0,4.155702e+07,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 27411...",31.66,68.34,"{'display_date': '2024-05-15', 'competitors': ...",131479890.0,0.0,"['Entertainment', 'Music', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
11096,weather.gov,533,113,98406201,87.0,2.0085,5.60,61.36,167395.0,2.453264e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 98406...",34.28,65.72,"{'display_date': '2024-05-15', 'competitors': ...",24584446.0,0.0,[],{'props': {'pageProps': {'page': {'actualDate'...


In [30]:
final_df[final_df.domain_name.duplicated()]

,domain_name,global_rank,country_rank,visits_no,authority_score,pages_per_visit,avg_visit_duration,bounce_rate,referring_domains,backlinks,traffic_by_country,traffic_by_device,desktop_device,mobile_device,competitors,organic_traffic,paid_traffic,categories,site_data
539,eurasiantimes.com,10649,1933,6957054,58.0,1.9905,1.29,33.63,20518.0,7.303760e+05,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 69570...",7.96,92.04,"{'display_date': '2024-05-15', 'competitors': ...",345584.0,0.0,['Airlines'],{'props': {'pageProps': {'page': {'actualDate'...
540,theaviationgeekclub.com,29747,7890,2281588,40.0,1.4124,2.21,73.90,13926.0,1.996770e+05,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22815...",3.47,96.53,"{'display_date': '2024-05-15', 'competitors': ...",42098.0,0.0,['Airlines'],{'props': {'pageProps': {'page': {'actualDate'...
542,spacex.com,26856,7951,2561996,65.0,2.2883,4.20,50.43,54783.0,5.911933e+06,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 25619...",39.18,60.82,"{'display_date': '2024-05-15', 'competitors': ...",1989065.0,0.0,['Airlines'],{'props': {'pageProps': {'page': {'actualDate'...
544,spaceflightnow.com,37989,8350,1735800,54.0,1.1995,3.09,85.16,28131.0,7.104026e+06,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 17358...",17.85,82.15,"{'display_date': '2024-05-15', 'competitors': ...",300414.0,0.0,['Airlines'],{'props': {'pageProps': {'page': {'actualDate'...
546,spacenews.com,36903,9740,1790464,54.0,1.3369,10.26,83.08,44908.0,1.006704e+07,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 17904...",27.94,72.06,"{'display_date': '2024-05-15', 'competitors': ...",299851.0,0.0,['Airlines'],{'props': {'pageProps': {'page': {'actualDate'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11093,forbes.com,283,106,170285922,100.0,1.4842,4.49,76.53,1508842.0,4.330822e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 17028...",30.32,69.68,"{'display_date': '2024-05-15', 'competitors': ...",98133930.0,541805.0,"['Newspapers', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
11094,adobe.com,95,109,436231535,100.0,3.1437,7.37,57.58,2013681.0,1.337587e+09,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 43623...",63.25,36.75,"{'display_date': '2024-05-15', 'competitors': ...",179383533.0,4320379.0,"['Computer Software and Development', 'Online ...",{'props': {'pageProps': {'page': {'actualDate'...
11095,genius.com,155,111,274113391,94.0,2.5551,8.19,69.83,251976.0,4.155702e+07,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 27411...",31.66,68.34,"{'display_date': '2024-05-15', 'competitors': ...",131479890.0,0.0,"['Entertainment', 'Music', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
11096,weather.gov,533,113,98406201,87.0,2.0085,5.60,61.36,167395.0,2.453264e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 98406...",34.28,65.72,"{'display_date': '2024-05-15', 'competitors': ...",24584446.0,0.0,[],{'props': {'pageProps': {'page': {'actualDate'...


In [33]:
df_cleaned = final_df.drop_duplicates(subset=['domain_name', 'global_rank'])

In [38]:
df_cleaned[df_cleaned.domain_name=='weather.gov']

,domain_name,global_rank,country_rank,visits_no,authority_score,pages_per_visit,avg_visit_duration,bounce_rate,referring_domains,backlinks,traffic_by_country,traffic_by_device,desktop_device,mobile_device,competitors,organic_traffic,paid_traffic,categories,site_data
2654,weather.gov,533,113,98406201,87.0,2.0085,5.6,61.36,167395.0,245326350.0,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 98406...",34.28,65.72,"{'display_date': '2024-05-15', 'competitors': ...",24584446.0,0.0,[],{'props': {'pageProps': {'page': {'actualDate'...


In [36]:
df_cleaned.reset_index(drop=True, inplace=True)

In [39]:
df_cleaned.isnull().sum()

domain_name            0
global_rank            0
country_rank           0
visits_no              0
authority_score        1
pages_per_visit        0
avg_visit_duration     0
bounce_rate            0
referring_domains      1
backlinks              1
traffic_by_country     0
traffic_by_device      0
desktop_device         0
mobile_device          0
competitors            0
organic_traffic       13
paid_traffic          13
categories             0
site_data              0
dtype: int64

In [40]:
df_cleaned[df_cleaned.isnull().any(axis=1)]

,domain_name,global_rank,country_rank,visits_no,authority_score,pages_per_visit,avg_visit_duration,bounce_rate,referring_domains,backlinks,traffic_by_country,traffic_by_device,desktop_device,mobile_device,competitors,organic_traffic,paid_traffic,categories,site_data
2345,motorola-mail.com,26507,9059,2600006,6.0,1.1584,2.55,87.24,403.0,598956.0,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 26000...",1.88,98.12,"{'display_date': '2024-05-15', 'competitors': []}",NaN,NaN,['Online Services'],{'props': {'pageProps': {'page': {'actualDate'...
4362,datasltn.com,868381,209164,33905,4.0,4.2155,24.63,15.19,76.0,155.0,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 33905...",36.46,63.54,"{'display_date': '2024-05-15', 'competitors': []}",NaN,NaN,[],{'props': {'pageProps': {'page': {'actualDate'...
4498,email-libertymutual.com,44066,9077,1456498,5.0,1.1811,2.05,87.67,177.0,926.0,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 14564...",2.88,97.12,"{'display_date': '2024-05-15', 'competitors': []}",NaN,NaN,['Insurance'],{'props': {'pageProps': {'page': {'actualDate'...
5355,walmart.net,164903,45436,289925,5.0,5.0507,13.70,31.73,398.0,28482.0,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 28992...",92.12,7.88,"{'display_date': '2024-05-15', 'competitors': []}",NaN,NaN,['Market Research'],{'props': {'pageProps': {'page': {'actualDate'...
5606,videobin.co,44886,15964,1426284,4.0,1.5021,4.99,74.07,2669.0,1181851.0,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 14262...",12.82,87.18,"{'display_date': '2024-05-15', 'competitors': []}",NaN,NaN,[],{'props': {'pageProps': {'page': {'actualDate'...
7487,mtbachelor.com,296908,66072,135843,NaN,2.0089,1.83,65.07,NaN,NaN,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 13584...",35.22,64.78,"{'display_date': '2024-05-15', 'competitors': []}",181263.0,3397.0,"['Hospitality', 'Travel and Tourism']",{'props': {'pageProps': {'page': {'actualDate'...
7576,indeed.net,24950,5693,2784015,4.0,5.1601,-132.23,62.81,208.0,803566.0,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 27840...",15.83,84.17,"{'display_date': '2024-05-15', 'competitors': []}",NaN,NaN,[],{'props': {'pageProps': {'page': {'actualDate'...
7622,interviewarea.com,61733,26612,977015,6.0,2.1814,1.98,14.31,1601.0,358721.0,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 97701...",30.98,69.02,"{'display_date': '2024-05-15', 'competitors': []}",NaN,NaN,[],{'props': {'pageProps': {'page': {'actualDate'...
7667,hwatchtvnow.co,167114,36916,284992,2.0,1.3786,8.41,87.54,148.0,316.0,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 28499...",100.00,0.00,"{'display_date': '2024-05-15', 'competitors': []}",NaN,NaN,[],{'props': {'pageProps': {'page': {'actualDate'...
7681,chumsearch.com,116059,5679,450667,5.0,1.5622,14.73,66.22,225.0,877.0,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 45066...",100.00,0.00,"{'display_date': '2024-05-15', 'competitors': []}",NaN,NaN,[],{'props': {'pageProps': {'page': {'actualDate'...


In [41]:
df_cleaned.to_csv('cleaned_sites_data.csv', index=False)

## Remove duplicates from sites list

In [43]:
sites_df = pd.read_csv('/content/semrush_listed_sites_concurrent.csv')
sites_df_cleaned = sites_df.drop_duplicates(subset=['domain_name', 'total_traffic'])

In [44]:
sites_df_cleaned

,domain_name,total_traffic,desktop_percent,desktop_traffic,mobile_percent,mobile_traffic,mom,yoy,top_source,category
0,uworld.com,10061449,97.620542,9822041,2.379458,239408,-0.960057,-7.524990,direct,accounting-and-auditing
1,sharefile.com,5157785,90.241470,4654461,9.758530,503324,-28.410870,1.497492,direct,accounting-and-auditing
2,freetaxusa.com,4706463,44.103396,2075710,55.896604,2630753,-75.268360,25.183290,direct,accounting-and-auditing
3,deloitte.com,3862722,86.295131,3333341,13.704869,529381,-1.286360,13.783615,direct,accounting-and-auditing
4,mip.com,2411077,7.075884,170605,92.924116,2240472,-12.416770,11387.501000,direct,accounting-and-auditing
...,...,...,...,...,...,...,...,...,...,...
11019,x.com,518788997,35.110294,182148342,64.889706,336640655,315.687070,136854.550000,direct,all
11024,chatgpt.com,393178599,73.731321,289895774,26.268679,103282825,60790.520000,7211.430700,direct,all
11039,asuracomic.net,266979941,2.140662,5715138,97.859338,261264803,42717.016000,NaN,direct,all
11057,temu.com,164876300,33.444413,55141910,66.555587,109734390,17.354336,196.099660,direct,all


In [45]:
df_cleaned

,domain_name,global_rank,country_rank,visits_no,authority_score,pages_per_visit,avg_visit_duration,bounce_rate,referring_domains,backlinks,traffic_by_country,traffic_by_device,desktop_device,mobile_device,competitors,organic_traffic,paid_traffic,categories,site_data
0,rainharvest.com,20,24,2234905088,100.0,3.1522,13.97,52.80,379479.0,1.994620e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22349...",80.18,19.82,"{'display_date': '2024-05-15', 'competitors': ...",395662082.0,1842.0,"['Entertainment', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
1,supplyone.com,20,24,2234905088,100.0,3.1522,13.97,52.80,379479.0,1.994620e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22349...",80.18,19.82,"{'display_date': '2024-05-15', 'competitors': ...",395662082.0,1842.0,"['Entertainment', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
2,reusabletranspack.com,20,24,2234905088,100.0,3.1522,13.97,52.80,379479.0,1.994620e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22349...",80.18,19.82,"{'display_date': '2024-05-15', 'competitors': ...",395662082.0,1842.0,"['Entertainment', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
3,nglantz.com,20,24,2234905088,100.0,3.1522,13.97,52.80,379479.0,1.994620e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22349...",80.18,19.82,"{'display_date': '2024-05-15', 'competitors': ...",395662082.0,1842.0,"['Entertainment', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
4,harcofittings.com,20,24,2234905088,100.0,3.1522,13.97,52.80,379479.0,1.994620e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22349...",80.18,19.82,"{'display_date': '2024-05-15', 'competitors': ...",395662082.0,1842.0,"['Entertainment', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9589,x.com,19,21,2336636293,69.0,5.3197,-38.59,52.19,515201.0,1.740284e+09,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 23366...",39.90,60.10,"{'display_date': '2024-05-15', 'competitors': ...",6070613.0,0.0,[],{'props': {'pageProps': {'page': {'actualDate'...
9590,chatgpt.com,0,0,2693601492,21.0,2.6345,-6573.64,50.09,8109.0,7.695370e+05,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 26936...",69.77,30.23,"{'display_date': '2024-05-15', 'competitors': ...",56.0,0.0,[],{'props': {'pageProps': {'page': {'actualDate'...
9591,asuracomic.net,0,0,352731752,19.0,4.2363,-3463.89,40.69,33.0,7.511000e+03,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 35273...",2.28,97.72,"{'display_date': '2024-05-15', 'competitors': ...",363841.0,0.0,[],{'props': {'pageProps': {'page': {'actualDate'...
9592,temu.com,81,65,531164293,77.0,5.9862,9.74,48.69,29259.0,2.406524e+07,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 53116...",35.05,64.95,"{'display_date': '2024-05-15', 'competitors': ...",24840067.0,5693248.0,[],{'props': {'pageProps': {'page': {'actualDate'...


In [46]:
merged_df = pd.merge(sites_df_cleaned, df_cleaned, on='domain_name', how='left', indicator=True)

In [47]:
merged_df

,domain_name,total_traffic,desktop_percent,desktop_traffic,mobile_percent,mobile_traffic,mom,yoy,top_source,category,global_rank,country_rank,visits_no,authority_score,pages_per_visit,avg_visit_duration,bounce_rate,referring_domains,backlinks,traffic_by_country,traffic_by_device,desktop_device,mobile_device,competitors,organic_traffic,paid_traffic,categories,site_data,_merge
0,uworld.com,10061449,97.620542,9822041,2.379458,239408,-0.960057,-7.524990,direct,accounting-and-auditing,5630.0,1311.0,1.294365e+07,61.0,4.4858,12.53,28.18,9925.0,3.710690e+05,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 12943...",95.70,4.30,"{'display_date': '2024-05-15', 'competitors': ...",725509.0,14709.0,"['Distance Learning', 'Education']",{'props': {'pageProps': {'page': {'actualDate'...,both
1,sharefile.com,5157785,90.241470,4654461,9.758530,503324,-28.410870,1.497492,direct,accounting-and-auditing,11406.0,2668.0,6.467355e+06,53.0,3.9965,7.52,31.85,25499.0,2.758737e+06,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 64673...",89.53,10.47,"{'display_date': '2024-05-15', 'competitors': ...",211659.0,30418.0,['Computer Software and Development'],{'props': {'pageProps': {'page': {'actualDate'...,both
2,freetaxusa.com,4706463,44.103396,2075710,55.896604,2630753,-75.268360,25.183290,direct,accounting-and-auditing,15201.0,2928.0,4.768177e+06,57.0,2.8993,12.89,49.60,5669.0,5.882900e+04,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 47681...",44.44,55.56,"{'display_date': '2024-05-15', 'competitors': ...",1128679.0,250332.0,['Finance'],{'props': {'pageProps': {'page': {'actualDate'...,both
3,deloitte.com,3862722,86.295131,3333341,13.704869,529381,-1.286360,13.783615,direct,accounting-and-auditing,6600.0,3540.0,1.115800e+07,74.0,3.6380,8.57,45.46,260460.0,4.980576e+07,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 11157...",73.49,26.51,"{'display_date': '2024-05-15', 'competitors': ...",4023193.0,83745.0,[],{'props': {'pageProps': {'page': {'actualDate'...,both
4,mip.com,2411077,7.075884,170605,92.924116,2240472,-12.416770,11387.501000,direct,accounting-and-auditing,20.0,24.0,2.234905e+09,100.0,3.1522,13.97,52.80,379479.0,1.994620e+08,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 22349...",80.18,19.82,"{'display_date': '2024-05-15', 'competitors': ...",395662082.0,1842.0,"['Entertainment', 'Online Services']",{'props': {'pageProps': {'page': {'actualDate'...,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9591,x.com,518788997,35.110294,182148342,64.889706,336640655,315.687070,136854.550000,direct,all,19.0,21.0,2.336636e+09,69.0,5.3197,-38.59,52.19,515201.0,1.740284e+09,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 23366...",39.90,60.10,"{'display_date': '2024-05-15', 'competitors': ...",6070613.0,0.0,[],{'props': {'pageProps': {'page': {'actualDate'...,both
9592,chatgpt.com,393178599,73.731321,289895774,26.268679,103282825,60790.520000,7211.430700,direct,all,0.0,0.0,2.693601e+09,21.0,2.6345,-6573.64,50.09,8109.0,7.695370e+05,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 26936...",69.77,30.23,"{'display_date': '2024-05-15', 'competitors': ...",56.0,0.0,[],{'props': {'pageProps': {'page': {'actualDate'...,both
9593,asuracomic.net,266979941,2.140662,5715138,97.859338,261264803,42717.016000,NaN,direct,all,0.0,0.0,3.527318e+08,19.0,4.2363,-3463.89,40.69,33.0,7.511000e+03,"{'display_date': '2024-05-01', 'traffic_by_cou...","{'display_date': '2024-05-01', 'visits': 35273...",2.28,97.72,"{'display_date': '2024-05-15', 'competitors': ...",363841.0,0.0,[],{'props': {'pageProps': {'page': {'actualDate'...,both
9594,temu.com,164876300,33.444413,55141910,66.555587,10

In [56]:
missing_in_df2 = merged_df[merged_df['_merge'] == 'left_only']
print("\nRows in df1 with missing 'domain_name' in df2:\n")
missing_in_df2


Rows in df1 with missing 'domain_name' in df2:



,domain_name,total_traffic,desktop_percent,desktop_traffic,mobile_percent,mobile_traffic,mom,yoy,top_source,category,global_rank,country_rank,visits_no,authority_score,pages_per_visit,avg_visit_duration,bounce_rate,referring_domains,backlinks,traffic_by_country,traffic_by_device,desktop_device,mobile_device,competitors,organic_traffic,paid_traffic,categories,site_data,_merge
1665,vlr.gg,7243511,53.140818,3849261,46.859182,3394250,-21.183624,-9.533933,direct,computer-and-video-games,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3184,maranastockyards.com,21543,0.000000,0,100.000000,21543,NaN,2538.461400,direct,farms-and-ranches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [49]:
missing_in_df2

,domain_name,total_traffic,desktop_percent,desktop_traffic,mobile_percent,mobile_traffic,mom,yoy,top_source,category,global_rank,country_rank,visits_no,authority_score,pages_per_visit,avg_visit_duration,bounce_rate,referring_domains,backlinks,traffic_by_country,traffic_by_device,desktop_device,mobile_device,competitors,organic_traffic,paid_traffic,categories,site_data,_merge
1665,vlr.gg,7243511,53.140818,3849261,46.859182,3394250,-21.183624,-9.533933,direct,computer-and-video-games,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3184,maranastockyards.com,21543,0.000000,0,100.000000,21543,NaN,2538.461400,direct,farms-and-ranches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [57]:
merged_df.drop('_merge', axis=1, inplace=True)

In [59]:
merged_df.to_csv('merged_sites_data.csv', index=False)

In [60]:
!cp /content/merged_sites_data.csv /content/drive/MyDrive/mldata/mylane